## Homework 3: Machine Learning for Classification

In [176]:
import pandas as pd
import numpy as np

# Load the dataset
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)

# Make everything lower case and change spaces to underscores (in both headings and data)
df.columns = df.columns.str.lower().str.replace(' ', '_')
strings = list(df.dtypes[df.dtypes == 'object'].index)
for col in strings:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


### Data preparation

In [177]:
# Check if the missing values are presented in the features.
missing_values = df.isnull().sum()
print("Missing values in each column:\n", missing_values)

# If there are missing values:
# For categorical features, replace them with 'NA'
df.fillna({'lead_source': 'NA', 'industry': 'NA', 'employment_status': 'NA', 'location': 'NA'}, inplace=True)
# For numerical features, replace with with 0.0
df.fillna({'annual_income': 0.0}, inplace=True)

Missing values in each column:
 lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64


### Question 1. What is the most frequent observation (mode) for the column industry?

In [178]:
industry_mode = df['industry'].mode()[0]
print("The most frequent observation (mode) for the column industry is:", industry_mode)

The most frequent observation (mode) for the column industry is: retail


### Question 2.

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.
What are the two features that have the biggest correlation?

In [179]:
numerical_features = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
df[numerical_features].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


What are the two features that have the biggest correlation?

interaction_count and lead_score 0.009888\
number_of_courses_viewed and lead_score -0.004879\
number_of_courses_viewed and interaction_count -0.023565\
annual_income and interaction_count 0.027036

### Split the data.

Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value y is not in your dataframe.

In [180]:
from sklearn.model_selection import train_test_split

np.random.seed(42)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

### Question 3.

Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only. Round the scores to 2 decimals using round(score, 2).

In [181]:
from sklearn.metrics import mutual_info_score

def calculate_mi(series):
    return mutual_info_score(series, y_train)

categorical_features = ['lead_source', 'industry', 'employment_status', 'location']
df_mi = df_train[categorical_features].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='mutual information')
display(df_mi)

,mutual information
lead_source,0.024803
employment_status,0.016345
industry,0.006161
location,0.001453


### Question 4. Now let's train a logistic regression.

In [182]:
from sklearn.feature_extraction import DictVectorizer

# Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
dv = DictVectorizer(sparse=False)
train_dict = df_train[categorical_features + numerical_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_features + numerical_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
# model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

#Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
accuracy_all_features = (converted_decision == y_val).mean()
#accuracy_all_features = round((converted_decision == y_val).mean(), 2)
print("The accuracy of the model using all features is:", accuracy_all_features)

The accuracy of the model using all features is: 0.6996587030716723


### Question 5.

Let's find the least useful feature using the feature elimination technique.\
Train a model using the same features and parameters as in Q4 (without rounding).  <-- Done above\
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.\
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

In [183]:
all_features = categorical_features + numerical_features

for current_feature in all_features:
    remaining_features = all_features.copy()
    remaining_features.remove(current_feature)

    # Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
    dv = DictVectorizer(sparse=False)
    train_dict = df_train[remaining_features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[remaining_features].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # Fit the model on the training dataset.
    # To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    # model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    #Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    accuracy_remaining_features = (converted_decision == y_val).mean()
    #accuracy_remaining_features = round((converted_decision == y_val).mean(), 2)
    print("Difference in the accuracy of the model using all features except", current_feature, "is:", accuracy_all_features - accuracy_remaining_features)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    #Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    accuracy_remaining_features = (converted_decision == y_val).mean()
    #accuracy_remaining_features = round((converted_decision == y_val).mean(), 2)
    print("Difference in the accuracy of the model using all features except", current_feature, "is:", accuracy_all_features - accuracy_remaining_features)

Difference in the accuracy of the model using all features except lead_source is: -0.0034129692832765013
Difference in the accuracy of the model using all features except lead_source is: -0.0034129692832765013
Difference in the accuracy of the model using all features except industry is: 0.0
Difference in the accuracy of the model using all features except industry is: 0.0
Difference in the accuracy of the model using all features except employment_status is: -0.0034129692832765013
Difference in the accuracy of the model using all features except employment_status is: -0.0034129692832765013
Difference in the accuracy of the model using all features except location is: 0.0
Difference in the accuracy of the model using all features except location is: 0.0
Difference in the accuracy of the model using all features except number_of_courses_viewed is: 0.11604095563139927
Difference in the accuracy of the model using all features except number_of_courses_viewed is: 0.11604095563139927
Differ

### Question 6.

Now let's train a regularized logistic regression.\
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].\
Train models using all the features as in Q4.\
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [ ]:
from sklearn.linear_model import Ridge

for r in [0.01, 0.1, 1, 10, 100]:
    # Initialize the Ridge model with a specific alpha value
    ridge_model = Ridge(alpha=r)

    # Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
    dv = DictVectorizer(sparse=False)
    train_dict = df_train[categorical_features + numerical_features].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[categorical_features + numerical_features].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # Fit the model on the training dataset.
    # To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    # model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    #Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    accuracy_all_features = (converted_decision == y_val).mean()
    #accuracy_all_features = round((converted_decision == y_val).mean(), 2)
    print("The accuracy of the model using all features is:", accuracy_all_features)

Using r-value: 0.01
Using r-value: 0.1
Using r-value: 1
Using r-value: 10
Using r-value: 100
